In [1]:
pip install dotenv

In [2]:
from dotenv import find_dotenv,load_dotenv

In [ ]:
import os

load_dotenv(override=True)  # Loads the variables from .env

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [5]:
# Sample Documents

documents = [
    "This is a list that contains sample documents.",
    "Keywords are important for keyword-based search.",
    "Document analysis involves extracting keywords.",
    "Keyword-based search relies on sparse embeddings."
]

In [6]:
# here we need to find how much similarity below query has with which documents
query = "keyword-based search"

#### Cleaning the Text

In [7]:
import re
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove Punctuation
    text = re.sub(r'[^\w\s]','', text)
    return text

#### Applying preprocessing on the documents

In [8]:
preprocessed_doc = [preprocess_text(doc) for doc in documents]
print(preprocessed_doc)

['this is a list that contains sample documents', 'keywords are important for keywordbased search', 'document analysis involves extracting keywords', 'keywordbased search relies on sparse embeddings']


## Sparse Vector Implementation(Keyword Based Search)

#### Applying tf idf vectorization on preprocessed documents and query(for sparse search)

In [9]:
preprocessed_query = preprocess_text(query)
preprocessed_query

'keywordbased search'

In [10]:
tfidf_obj = TfidfVectorizer()
vectorized_doc = tfidf_obj.fit_transform(preprocessed_doc)

In [11]:
vectorized_doc_arr = vectorized_doc.toarray()
vectorized_doc_arr

array([[0.        , 0.        , 0.37796447, 0.        , 0.37796447,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.37796447, 0.        , 0.        , 0.37796447, 0.        ,
        0.        , 0.37796447, 0.        , 0.        , 0.37796447,
        0.37796447],
       [0.        , 0.4533864 , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.4533864 , 0.4533864 , 0.        ,
        0.        , 0.35745504, 0.35745504, 0.        , 0.        ,
        0.        , 0.        , 0.35745504, 0.        , 0.        ,
        0.        ],
       [0.46516193, 0.        , 0.        , 0.46516193, 0.        ,
        0.        , 0.46516193, 0.        , 0.        , 0.46516193,
        0.        , 0.        , 0.36673901, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.43671931, 0.        , 0.        , 0.       

In [12]:
vectorized_doc_arr[0]

array([0.        , 0.        , 0.37796447, 0.        , 0.37796447,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.37796447, 0.        , 0.        , 0.37796447, 0.        ,
       0.        , 0.37796447, 0.        , 0.        , 0.37796447,
       0.37796447])

In [13]:
preprocessed_query

'keywordbased search'

In [14]:
vectorized_query = tfidf_obj.transform([preprocessed_query])
vectorized_query

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2 stored elements and shape (1, 21)>

In [15]:
vectorized_query_arr = vectorized_query.toarray()

In [16]:
vectorized_query_arr

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.70710678, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.70710678, 0.        , 0.        ,
        0.        ]])

#### Finding the Similarity between the query and the documents

In [17]:
similarity = cosine_similarity(vectorized_doc_arr,vectorized_query_arr)

In [18]:
print(similarity)

[[0.        ]
 [0.50551777]
 [0.        ]
 [0.48693426]]


In [19]:
preprocessed_query

'keywordbased search'

In [20]:
preprocessed_doc

['this is a list that contains sample documents',
 'keywords are important for keywordbased search',
 'document analysis involves extracting keywords',
 'keywordbased search relies on sparse embeddings']

#### Ranking of the documents using similarity

In [21]:
ranked_indices = np.argsort(similarity,axis=0)[::-1].flatten()

In [22]:
ranked_indices

array([1, 3, 2, 0])

In [23]:
ranked_doc = [preprocessed_doc[i] for i in ranked_indices]

In [24]:
for i, doc in enumerate(ranked_doc):
    print(f"Rank {i+1} -> {doc}")

Rank 1 -> keywords are important for keywordbased search
Rank 2 -> keywordbased search relies on sparse embeddings
Rank 3 -> document analysis involves extracting keywords
Rank 4 -> this is a list that contains sample documents


## Dense Vector Implementation (Manual)

In [25]:
document_embedding = np.array([
    [0.634, 0.234, 0.867, 0.042, 0.249],
    [0.123, 0.456, 0.789, 0.321, 0.654],
    [0.987, 0.654, 0.321, 0.123, 0.456]
])

In [26]:
document_embedding

array([[0.634, 0.234, 0.867, 0.042, 0.249],
       [0.123, 0.456, 0.789, 0.321, 0.654],
       [0.987, 0.654, 0.321, 0.123, 0.456]])

In [27]:
query_embedding = np.array([[0.789, 0.321, 0.654, 0.987, 0.123]])

In [28]:
query_embedding

array([[0.789, 0.321, 0.654, 0.987, 0.123]])

In [29]:
similarities = cosine_similarity(document_embedding,query_embedding)

In [30]:
similarities

array([[0.73558979],
       [0.67357898],
       [0.71517305]])

In [31]:
ranked_indices_dense = np.argsort(similarities,axis=0)[::-1].flatten()

In [32]:
ranked_indices_dense

array([0, 2, 1])

In [33]:
for i, idx in enumerate(ranked_indices_dense):
    print(f"Rank {i+1}: Document {idx+1}")

Rank 1: Document 1
Rank 2: Document 3
Rank 3: Document 2


### Using Sentence Transformers for Dense Embedding

# Using Langchain

In [34]:
pip install langchain langchain-core langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.1 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.3.44
    Uninstalling langsmith-0.3.44:
      Successfully uninstalled langsmith-0.3.44
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.63
    Uninstalling langchain-core-0.3.63:
      Successfully uninstalled langchain-core-0.3.63


In [35]:
pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 4.1 MB/s eta 0:00:00


In [36]:
from langchain_community.document_loaders import PyPDFLoader

In [38]:
loader = PyPDFLoader('/content/sample_data/Machine_Learning_Essay.pdf')
documents = loader.load()
print(documents)

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250410052841', 'source': '/content/sample_data/Machine_Learning_Essay.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Machine Learning: Transforming the Future of Technology\nMachine Learning (ML) is a dynamic and rapidly evolving branch of artificial intelligence (AI) that\nenables systems to learn from data, identify patterns, and make decisions with minimal human\nintervention. It has revolutionized how we interact with technology, contributing to advancements in\nnumerous fields such as healthcare, finance, transportation, and entertainment.\nAt its core, machine learning is about developing algorithms that allow computers to learn from and\nmake predictions or decisions based on data. These algorithms improve over time as they are\nexposed to more data, enhancing their accuracy and efficiency. ML is categorized into three main\ntypes: supervis

In [39]:
documents[0].page_content

'Machine Learning: Transforming the Future of Technology\nMachine Learning (ML) is a dynamic and rapidly evolving branch of artificial intelligence (AI) that\nenables systems to learn from data, identify patterns, and make decisions with minimal human\nintervention. It has revolutionized how we interact with technology, contributing to advancements in\nnumerous fields such as healthcare, finance, transportation, and entertainment.\nAt its core, machine learning is about developing algorithms that allow computers to learn from and\nmake predictions or decisions based on data. These algorithms improve over time as they are\nexposed to more data, enhancing their accuracy and efficiency. ML is categorized into three main\ntypes: supervised learning, unsupervised learning, and reinforcement learning.\nSupervised learning involves training a model on a labeled dataset, where the input data is paired\nwith the correct output. The model learns to map inputs to the correct output and is then te

In [40]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [41]:
splitter = RecursiveCharacterTextSplitter(chunk_size=100,chunk_overlap=30)
chunks = splitter.split_documents(documents)
print(chunks)

[Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250410052841', 'source': '/content/sample_data/Machine_Learning_Essay.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Machine Learning: Transforming the Future of Technology'), Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250410052841', 'source': '/content/sample_data/Machine_Learning_Essay.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Machine Learning (ML) is a dynamic and rapidly evolving branch of artificial intelligence (AI) that'), Document(metadata={'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250410052841', 'source': '/content/sample_data/Machine_Learning_Essay.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='enables systems to learn from data, identify patterns, and make de

In [42]:
len(chunks)

44

In [43]:
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
openai_api_key = "your api key -> better to give in a .env file or set an environment variable"
langsmith_key = "fetch from .env file or environment variables"

In [45]:
embedding = OpenAIEmbeddings(api_key=openai_api_key,model="text-embedding-3-large")

<ipython-input-45-3405659616>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings(api_key=openai_api_key,model="text-embedding-3-large")


In [46]:
from langchain.vectorstores import Chroma

In [47]:
pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.8 MB/s eta 0

In [48]:
vector_store = Chroma.from_documents(chunks,embedding)

In [49]:
vectorstore_retriever = vector_store.as_retriever(search_kwargs={"k":3})

In [50]:
vectorstore_retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x7df57cb901d0>, search_kwargs={'k': 3})

## We need keyword based also

In [51]:
pip install rank_bm25

In [52]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

In [53]:
keyword_retriever = BM25Retriever.from_documents(chunks)

## Ensemble Retriever

In [54]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retriever,keyword_retriever],weights=[0.7,0.3])

In [55]:
model_name = "HuggingFaceH4/zephyr-7b-beta"

In [56]:
pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 21.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [57]:
pip install accelerate

In [58]:
import torch

In [59]:
pip install transformers

In [60]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain import HuggingFacePipeline

In [61]:
def load_quantized_model(model_name: str):
  """
  model_name :- Name or path of the model to be loaded
  return: Loaded quantized model
  """
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16
  )
  model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, torch_dtype=torch.bfloat16)
  return model

In [62]:
def initialize_tokenizer(model_name: str):
  """
  model_name :- Name or path of the model to be loaded for tokenizer initialization
  return: Initialized tokenizer
  """

  tokenizer = AutoTokenizer.from_pretrained(model_name,return_token_type_ids=False)
  tokenizer.bos_token_id = tokenizer.eos_token_id
  return tokenizer

In [ ]:
HUGGINGFACEHUB_API_TOKEN="<Your hugging face token>"

In [68]:
tokenizer = initialize_tokenizer(model_name)

In [ ]:
model = load_quantized_model(model_name)

In [66]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1
)


NameError: name 'model' is not defined

In [69]:
llm=HuggingFacePipeline(pipeline=pipeline)

<ipython-input-69-1712356630>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm=HuggingFacePipeline(pipeline=pipeline)


In [70]:
from langchain.chains import RetrievalQA

In [71]:
normal_chain=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=vectorstore_retriever)

In [72]:
hybrid_chain=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=ensemble_retriever)